In [ ]:
import os

if os.name == 'nt':
    measuring_root = "D:/measuring"
else:
    measuring_root = "/Users/jesse/Thesis/Code"
    
execfile(os.path.join(measuring_root, "analysis/scripts/setup_analysis.py"))
import analysis.lib.purification.purify_delayfb as pu_delayfb
from analysis.lib.tools import toolbox
reload(pu_delayfb)
%matplotlib inline

In [ ]:
couplings_both = dict()
calc_LDE_phases_both = dict()
LDE_phase_data_both = dict()

none_phases_both = dict()
none_phases_u_both = dict()
pi_phases_both = dict()
pi_phases_u_both = dict()
pi2_phases_both = dict()
pi2_phases_u_both = dict()

none_fit_both = dict()
pi_fit_both = dict()
pi2_fit_both = dict()

In [ ]:
import measurement.scripts.lt4_scripts.setup.msmt_params as msmt_params
reload(msmt_params)

carbons_both = {
    "_p1": [1,2,4,5,3,7],
    "_m1": [1,2,3,4,5,6,7],
} 

trans = "_m1"
carbons = carbons_both[trans]

LDE_length = 7e-6
dec_time = 2.256e-6 * 2
ms0_time = LDE_length - dec_time

couplings = []
calc_LDE_phases = []

print("LDE length (s): \t\t\t\t%e" % LDE_length)
print("Decoupling time (s): \t\t\t\t%e" % dec_time)
print("ms = 0 time (LDE length - dec. time) (s): \t%e" % ms0_time)
print("")

for c in carbons:
    print ("Carbon: %d" % c)
    ms0_freq = msmt_params.cfg['samples']['111no2']['C%d_freq_0' % c]
    ms1_freq = msmt_params.cfg['samples']['111no2']['C%d_freq_1%s' % (c, trans)]
    avg_freq = msmt_params.cfg['samples']['111no2']['C%d_freq%s' % (c, trans)]
    
    parallel_coupling = ms1_freq - ms0_freq
    print("ms = 0 frequency (kHz): \t\t%.3f" % (ms0_freq * 1e-3))
    print("ms = -1 frequency (kHz): \t\t%.3f" % (ms1_freq * 1e-3))
    print("frequency difference (kHz): \t\t%.3f" % (parallel_coupling*1e-3))
    
    phase = ms0_time * ms0_freq + dec_time * avg_freq
    print("calc. LDE phase (full rotations): \t%.3f" % phase)
    phase_deg = (phase % 1.0) * 360.0
    print("calc. LDE phase (degrees): \t\t%.3f" % phase_deg)
    
    calc_LDE_phases.append(phase_deg)
    
    couplings.append(parallel_coupling)
    
couplings = np.array(couplings) * 1e-3
calc_LDE_phases = np.array(calc_LDE_phases)

couplings_both[trans] = couplings
calc_LDE_phases_both[trans] = calc_LDE_phases

In [ ]:
reload(pu_delayfb)

pulses = ["_pi2_", "_pi_", "_none_"]
# nt = "20170804080000"
# ot = "20170805080000"
nt = "20170707170000"
ot = None
sections = [0, 1]

decays = [20] + [100] * 6

LDE_phase_data = dict()

freq_guess = calc_LDE_phases / 360.0

p = 4e-6

for i_c, c in enumerate(carbons):
    c_key = c
    LDE_phase_data[c_key] = dict()
    
    for pls in pulses:
        pls_key = pls.replace("_","")
        LDE_phase_data[c_key][pls_key] = None
        
        print("Processing: C%d, pulse %s" % (c, pls_key))
        
        m_name = "LDE_phase%s%sC%d_pow%duW_sec" % (trans if 'm1' not in trans else '', pls, c, int(p*1e6))
        data = dict()
        data["x"], data["y"], data["y_u"], data["fr"] = \
        pu_delayfb.calibrate_LDE_phase_stitched(
            do_fit=True,
            freq=freq_guess[i_c],
            decay=decays[i_c],
            fixed=[] if 'p1' in trans else [1],
            show_guess=True,
            multi_contains=[m_name + str(sec) for sec in sections],
            ret_data_fit=True
        )
        LDE_phase_data[c_key][pls_key] = data
        
LDE_phase_data_both[trans] = LDE_phase_data

In [ ]:
none_phases = np.zeros((len(carbons)))
none_phases_u = np.zeros_like(none_phases)
pi_phases = np.zeros_like(none_phases)
pi_phases_u = np.zeros_like(none_phases)
pi2_phases = np.zeros_like(none_phases)
pi2_phases_u = np.zeros_like(none_phases)

for i_c, c in enumerate(carbons):
    none_phases[i_c] = LDE_phase_data[c]['none']['fr']['acq_phase_per_rep']
    none_phases_u[i_c] = LDE_phase_data[c]['none']['fr']['u_acq_phase_per_rep']
    
    pi_phases[i_c] = LDE_phase_data[c]['pi']['fr']['acq_phase_per_rep']
    pi_phases_u[i_c] = LDE_phase_data[c]['pi']['fr']['u_acq_phase_per_rep']
    
    pi2_phases[i_c] = LDE_phase_data[c]['pi2']['fr']['acq_phase_per_rep']
    pi2_phases_u[i_c] = LDE_phase_data[c]['pi2']['fr']['u_acq_phase_per_rep']

if 'p1' in trans:
    ombatterijen = [0,4]

    none_phases[ombatterijen] -= 360.0
    pi_phases[ombatterijen] -= 360.0
    pi2_phases[ombatterijen] -= 360.0
    calc_LDE_phases[ombatterijen] -= 360.0
    
pi_phases_both[trans] = pi_phases
none_phases_both[trans] = none_phases
pi2_phases_both[trans] = pi2_phases

pi_phases_u_both[trans] = pi_phases_u
none_phases_u_both[trans] = none_phases_u
pi2_phases_u_both[trans] = pi2_phases_u

In [ ]:
from analysis.lib.fitting import fit, common

none_fit = fit.fit1d(couplings, none_phases, common.fit_line, 1.0, 1.0, do_print=True, fixed=[], ret=True)
pi_fit = fit.fit1d(couplings, pi_phases, common.fit_line, 1.0, 1.0, do_print=True, fixed=[], ret=True)
pi2_fit = fit.fit1d(couplings, pi2_phases, common.fit_line, 1.0, 1.0, do_print=True, fixed=[], ret=True)

none_fit_both[trans] = none_fit
pi_fit_both[trans] = pi_fit
pi2_fit_both[trans] = pi2_fit

In [ ]:
plt.figure(figsize=(10,10))

e_pi2 = plt.errorbar(couplings, pi2_phases, pi2_phases_u, 
                     label="pi/2, a=%.1f +/- %.1f, b=%.3f +/- %.3f" % (
                        pi2_fit['params_dict']['a'], pi2_fit['error_dict']['a'],
                        pi2_fit['params_dict']['b'], pi2_fit['error_dict']['b']
                    ),  
                    fmt='o')
e_pi = plt.errorbar(couplings, pi_phases, pi_phases_u, 
                    label="pi (no repump), a=%.1f +/- %.1f, b=%.3f +/- %.3f" % (
                        pi_fit['params_dict']['a'], pi_fit['error_dict']['a'],
                        pi_fit['params_dict']['b'], pi_fit['error_dict']['b']
                    ), 
                    fmt='o')
e_none = plt.errorbar(couplings, none_phases, none_phases_u, 
                      label="none (always repump), a=%.1f +/- %.1f, b=%.3f +/- %.3f" % (
                        none_fit['params_dict']['a'], none_fit['error_dict']['a'],
                        none_fit['params_dict']['b'], none_fit['error_dict']['b']
                    ), 
                      fmt='o')
e_calc = plt.scatter(couplings, calc_LDE_phases, label="calculated", marker='x', zorder=100)

plot_xvals = np.linspace(-50,400)

plt.plot(plot_xvals, pi2_fit['fitfunc'](plot_xvals), color=e_pi2[0].get_color())
plt.plot(plot_xvals, pi_fit['fitfunc'](plot_xvals), color=e_pi[0].get_color())
plt.plot(plot_xvals, none_fit['fitfunc'](plot_xvals), color=e_none[0].get_color())

plt.legend()
plt.xlabel("coupling strength (kHz)")
plt.ylabel("LDE element phase (degrees)")
plt.ylim(-20,120)
plt.xlim(-60,100)

In [ ]:
plt.figure(figsize=(10,10))

transitions = ['_p1', '_m1']
tr_labels = ['+1', '-1']

for i_tr, tr in enumerate(transitions):
    e_pi2 = plt.errorbar(couplings_both[tr], pi2_phases_both[tr], pi2_phases_u_both[tr], 
                         label="pi/2 on %s, a=%.1f +/- %.1f, b=%.3f +/- %.3f" % (
                            tr_labels[i_tr],
                            pi2_fit_both[tr]['params_dict']['a'], pi2_fit_both[tr]['error_dict']['a'],
                            pi2_fit_both[tr]['params_dict']['b'], pi2_fit_both[tr]['error_dict']['b']
                        ),  
                        fmt='o')
    e_pi = plt.errorbar(couplings_both[tr], pi_phases_both[tr], pi_phases_u_both[tr], 
                        label="pi (no repump) on %s, a=%.1f +/- %.1f, b=%.3f +/- %.3f" % (
                            tr_labels[i_tr],
                            pi_fit_both[tr]['params_dict']['a'], pi_fit_both[tr]['error_dict']['a'],
                            pi_fit_both[tr]['params_dict']['b'], pi_fit_both[tr]['error_dict']['b']
                        ), 
                        fmt='o')
    e_none = plt.errorbar(couplings_both[tr], none_phases_both[tr], none_phases_u_both[tr], 
                          label="none (always repump) on %s, a=%.1f +/- %.1f, b=%.3f +/- %.3f" % (
                              tr_labels[i_tr],
                            none_fit_both[tr]['params_dict']['a'], none_fit_both[tr]['error_dict']['a'],
                            none_fit_both[tr]['params_dict']['b'], none_fit_both[tr]['error_dict']['b']
                        ), 
                          fmt='o')
    e_calc = plt.scatter(couplings_both[tr], calc_LDE_phases_both[tr], label="calculated on %s" % tr_labels[i_tr], marker='x', zorder=100, color='k')

# plot_xvals = np.linspace(-400,400)

# plt.plot(plot_xvals, pi2_fit['fitfunc'](plot_xvals), color=e_pi2[0].get_color())
# plt.plot(plot_xvals, pi_fit['fitfunc'](plot_xvals), color=e_pi[0].get_color())
# plt.plot(plot_xvals, none_fit['fitfunc'](plot_xvals), color=e_none[0].get_color())

plt.legend()
plt.xlabel("coupling strength (kHz)")
plt.ylabel("LDE element phase (degrees)")
plt.xlim(-75,90)
plt.ylim(-20,110)